# T20 Linear Regression Model Evaluation

This notebook provides comprehensive evaluation of the trained T20 linear regression model including detailed performance analysis, visualizations, and model diagnostics.

## Objectives
1. Load trained model from MLflow registry
2. Comprehensive performance evaluation
3. Create detailed visualizations
4. Analyze model strengths and limitations
5. Generate model evaluation report
6. Provide recommendations for improvements

In [ ]:
# Import required libraries
import mlflow
import mlflow.sklearn
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Import project modules
from cricket.ml.evaluation import T20ModelEvaluator
from cricket.ml.models.linear_regression import T20LinearRegression
from cricket.ml.training import T20TrainingPipeline

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")
print(f"MLflow version: {mlflow.__version__}")

## 1. Load Training Results and Model

In [ ]:
# Load training summary if available
try:
    with open("training_summary.json", "r") as f:
        training_summary = json.load(f)
    print("✅ Loaded training summary from previous notebook")
    print(f"   • Model: {training_summary['model_name']}")
    print(f"   • Test R²: {training_summary['test_r2']:.3f}")
    print(f"   • Test RMSE: {training_summary['test_rmse']:.1f} runs")
except FileNotFoundError:
    print("⚠️ Training summary not found. Please run t20_model_training.ipynb first.")
    # Set default values
    training_summary = {
        "model_name": "t20_runs_predictor",
        "experiment_name": "male_team_level_t20"
    }

# MLflow configuration
MLFLOW_TRACKING_URI = "sqlite:///../mlflow_setup/mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

MODEL_NAME = training_summary["model_name"]
EXPERIMENT_NAME = training_summary.get("experiment_name", "male_team_level_t20")

print(f"\nMLflow Configuration:")
print(f"   • Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   • Model Name: {MODEL_NAME}")
print(f"   • Experiment: {EXPERIMENT_NAME}")

In [ ]:
# Load the trained model from MLflow registry
try:
    model_uri = f"models:/{MODEL_NAME}/latest"
    loaded_sklearn_model = mlflow.sklearn.load_model(model_uri)
    
    # Create our wrapper model instance
    trained_model = T20LinearRegression()
    trained_model.model = loaded_sklearn_model
    trained_model._is_trained = True
    
    print(f"✅ Model loaded successfully from: {model_uri}")
    print(f"   • Model type: {type(loaded_sklearn_model).__name__}")
    print(f"   • Features: {trained_model.feature_names}")
    print(f"   • Coefficients: {loaded_sklearn_model.coef_}")
    print(f"   • Intercept: {loaded_sklearn_model.intercept_:.3f}")
    
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    print("Please ensure the model training notebook has been run and the model is registered.")
    raise

## 2. Recreate Test Data for Evaluation

In [ ]:
# Since we need test data for evaluation, let's recreate it using the same pipeline
# This is not ideal - in practice, we should save test data during training

DATA_PATH = "../data/ball_level_data.parquet"

print("🔄 Recreating test data for evaluation...")
print("Note: This recreates the data split - in production, test data should be saved during training.")

# Initialize pipeline with same parameters as training
eval_pipeline = T20TrainingPipeline(
    data_path=DATA_PATH,
    
    scaling_method="standard"
)

# Load and prepare data (same as training pipeline)
raw_data = eval_pipeline._load_data()
clean_data = eval_pipeline._prepare_t20_data(raw_data)
features_data = eval_pipeline._add_match_features(clean_data)
target_df, feature_samples = eval_pipeline._create_modeling_data(features_data)

# Split data (same ratios as training)
train_data, val_data, test_data = eval_pipeline._split_data(
    feature_samples, target_df, 0.7, 0.15, 0.15
)

# Prepare test features and targets
test_prep = eval_pipeline.feature_engineer.prepare_features(test_data)
X_test = eval_pipeline.feature_engineer.fit_transform(test_prep)  # Note: Should use transform only
y_test = eval_pipeline._get_targets_for_features(test_data, target_df)

print(f"✅ Test data recreated: {len(X_test)} samples")
print(f"   • Features shape: {X_test.shape}")
print(f"   • Target range: {y_test.min():.0f} - {y_test.max():.0f} runs")
print(f"   • Mean target: {y_test.mean():.1f} runs")

## 3. Initialize Model Evaluator

In [ ]:
# Create model evaluator
evaluator = T20ModelEvaluator(
    model=trained_model,
    X_test=X_test,
    y_test=y_test,
    feature_names=["current_score", "wickets_fallen", "overs_remaining"]
)

print(f"✅ Model evaluator initialized")
print(f"   • Test samples: {len(y_test)}")
print(f"   • Features: {evaluator.feature_names}")
print(f"   • Predictions computed: {len(evaluator.y_pred)}")

# Quick preview of predictions vs actuals
print(f"\n📊 Preview - First 10 predictions:")
print(f"{'Actual':<10} {'Predicted':<10} {'Error':<10} {'Abs Error':<10}")
print("-" * 45)
for i in range(min(10, len(y_test))):
    actual = y_test[i]
    predicted = evaluator.y_pred[i]
    error = actual - predicted
    abs_error = abs(error)
    print(f"{actual:<10.0f} {predicted:<10.0f} {error:<10.1f} {abs_error:<10.1f}")

## 4. Comprehensive Performance Metrics

In [ ]:
# Calculate comprehensive metrics
comprehensive_metrics = evaluator.calculate_comprehensive_metrics()

print("📊 COMPREHENSIVE PERFORMANCE METRICS")
print("=" * 50)

print(f"\n🎯 Core Regression Metrics:")
print(f"   • R² Score:           {comprehensive_metrics['r2_score']:.4f}")
print(f"   • Adjusted R²:        {comprehensive_metrics['adjusted_r2']:.4f}")
print(f"   • RMSE:              {comprehensive_metrics['rmse']:.2f} runs")
print(f"   • MAE:               {comprehensive_metrics['mae']:.2f} runs")
print(f"   • MAPE:              {comprehensive_metrics['mape']:.2f}%")

print(f"\n📏 Error Analysis:")
print(f"   • Max Error:          {comprehensive_metrics['max_error']:.1f} runs")
print(f"   • Mean Error:         {comprehensive_metrics['mean_error']:.2f} runs")
print(f"   • Std of Residuals:   {comprehensive_metrics['std_residuals']:.2f} runs")

print(f"\n🎯 Prediction Accuracy:")
print(f"   • Within 10 runs:     {comprehensive_metrics['within_10_runs']:.1f}%")
print(f"   • Within 20 runs:     {comprehensive_metrics['within_20_runs']:.1f}%")
print(f"   • Within 30 runs:     {comprehensive_metrics['within_30_runs']:.1f}%")

print(f"\n⚖️ Bias Analysis:")
print(f"   • Underestimate Rate: {comprehensive_metrics['underestimate_rate']:.1f}%")
print(f"   • Overestimate Rate:  {comprehensive_metrics['overestimate_rate']:.1f}%")

# Performance quality assessment
r2 = comprehensive_metrics['r2_score']
rmse = comprehensive_metrics['rmse']
within_20 = comprehensive_metrics['within_20_runs']

print(f"\n✅ Overall Assessment:")
if r2 >= 0.8:
    r2_quality = "Excellent"
elif r2 >= 0.7:
    r2_quality = "Good"
elif r2 >= 0.5:
    r2_quality = "Fair"
else:
    r2_quality = "Poor"

print(f"   • Model Quality: {r2_quality} (R² = {r2:.3f})")

if within_20 >= 80:
    accuracy_quality = "Excellent"
elif within_20 >= 70:
    accuracy_quality = "Good"
elif within_20 >= 60:
    accuracy_quality = "Fair"
else:
    accuracy_quality = "Poor"

print(f"   • Prediction Accuracy: {accuracy_quality} ({within_20:.1f}% within 20 runs)")

bias = abs(comprehensive_metrics['mean_error'])
if bias <= 2:
    bias_quality = "Excellent (Unbiased)"
elif bias <= 5:
    bias_quality = "Good (Low Bias)"
else:
    bias_quality = "Fair (Some Bias)"

print(f"   • Model Bias: {bias_quality} (Mean Error = {comprehensive_metrics['mean_error']:.2f})")

## 5. Model Visualization - Predictions vs Actual

In [ ]:
# Create predictions vs actual plot
fig_pred_actual = evaluator.plot_predictions_vs_actual(figsize=(12, 10))
plt.show()

print("📈 Predictions vs Actual Analysis:")
print(f"   • Points close to red line indicate good predictions")
print(f"   • Scatter pattern suggests model performance")
print(f"   • R² = {comprehensive_metrics['r2_score']:.3f} shows {comprehensive_metrics['r2_score']*100:.1f}% variance explained")

## 6. Residual Analysis

In [ ]:
# Create residual analysis plots
fig_residuals = evaluator.plot_residuals(figsize=(15, 6))
plt.show()

print("📊 Residual Analysis Insights:")
print(f"   • Left plot: Residuals vs Predicted - should show random scatter around 0")
print(f"   • Right plot: Residual distribution - should be approximately normal")
print(f"   • Mean residual: {comprehensive_metrics['mean_error']:.2f} (close to 0 is good)")
print(f"   • Std residual: {comprehensive_metrics['std_residuals']:.2f} runs")

# Check for patterns in residuals
if abs(comprehensive_metrics['mean_error']) <= 2:
    print(f"   ✅ No significant bias detected")
else:
    bias_direction = "overestimating" if comprehensive_metrics['mean_error'] < 0 else "underestimating"
    print(f"   ⚠️ Model may be {bias_direction} scores")

## 7. Feature Importance Analysis

In [ ]:
# Create feature importance plot
fig_importance = evaluator.plot_feature_importance(figsize=(12, 8))
plt.show()

# Get and display feature importance
importance_df = evaluator.model.get_feature_importance()
print("🔍 FEATURE IMPORTANCE ANALYSIS:")
print("=" * 40)

for _, row in importance_df.iterrows():
    feature = row['feature']
    coeff = row['coefficient']
    abs_coeff = row['abs_coefficient']
    
    if feature == 'intercept':
        print(f"\n📊 {feature.upper()}:")
        print(f"   • Value: {coeff:.3f}")
        print(f"   • Interpretation: Base prediction when all features are 0")
    else:
        direction = "increases" if coeff > 0 else "decreases"
        print(f"\n📊 {feature.upper().replace('_', ' ')}:")
        print(f"   • Coefficient: {coeff:.3f}")
        print(f"   • Impact: Each unit increase {direction} final score by {abs(coeff):.3f} runs")
        
        if feature == "current_score":
            print(f"   • Meaning: Higher current score → higher final total")
        elif feature == "wickets_fallen":
            if coeff < 0:
                print(f"   • Meaning: More wickets lost → lower final total")
            else:
                print(f"   • Meaning: More wickets lost → higher final total (unexpected!)")
        elif feature == "overs_remaining":
            if coeff > 0:
                print(f"   • Meaning: More overs left → higher scoring potential")
            else:
                print(f"   • Meaning: More overs left → lower final total (unexpected!)")

# Model equation
equation = evaluator.model.get_model_equation()
print(f"\n🧮 MODEL EQUATION:")
print(f"   {equation}")

## 8. Comprehensive Error Analysis

In [ ]:
# Create comprehensive error analysis
fig_error_analysis = evaluator.plot_error_analysis(figsize=(15, 12))
plt.show()

print("🔍 ERROR ANALYSIS INSIGHTS:")
print("=" * 40)

print(f"\n📊 Error Patterns:")
print(f"   • Top-left: Absolute errors vs actual scores")
print(f"   • Top-right: Error distribution across score ranges")
print(f"   • Bottom-left: Percentage errors vs actual scores")
print(f"   • Bottom-right: Cumulative error distribution")

# Analyze error patterns by score range
score_ranges = [(0, 120), (120, 140), (140, 160), (160, 180), (180, 250)]
abs_errors = np.abs(evaluator.residuals)

print(f"\n📈 Error Analysis by Score Range:")
for low, high in score_ranges:
    mask = (evaluator.y_test >= low) & (evaluator.y_test < high)
    if np.any(mask):
        range_errors = abs_errors[mask]
        range_count = len(range_errors)
        range_mean_error = np.mean(range_errors)
        print(f"   • {low}-{high} runs ({range_count} samples): Avg error = {range_mean_error:.1f} runs")

# Best and worst predictions
best_idx = np.argmin(abs_errors)
worst_idx = np.argmax(abs_errors)

print(f"\n🏆 Best Prediction:")
print(f"   • Actual: {evaluator.y_test[best_idx]:.0f} runs")
print(f"   • Predicted: {evaluator.y_pred[best_idx]:.0f} runs")
print(f"   • Error: {evaluator.residuals[best_idx]:.1f} runs")

print(f"\n😞 Worst Prediction:")
print(f"   • Actual: {evaluator.y_test[worst_idx]:.0f} runs")
print(f"   • Predicted: {evaluator.y_pred[worst_idx]:.0f} runs")
print(f"   • Error: {evaluator.residuals[worst_idx]:.1f} runs")

## 9. Model Report Generation

In [ ]:
# Generate comprehensive model report
model_report = evaluator.create_model_report()

print("📋 COMPREHENSIVE MODEL EVALUATION REPORT")
print("=" * 55)

# Performance Summary
performance = model_report['performance_summary']
print(f"\n🎯 PERFORMANCE SUMMARY:")
print(f"   • Overall Performance: {performance['overall_performance']}")
print(f"   • Prediction Accuracy: {performance['prediction_accuracy']}")
print(f"   • Bias Assessment: {performance['bias_assessment']}")

if performance['key_insights']:
    print(f"\n💡 Key Insights:")
    for insight in performance['key_insights']:
        print(f"   • {insight}")

# Data Summary
data_summary = model_report['data_summary']
print(f"\n📊 DATA SUMMARY:")
print(f"   • Test Samples: {data_summary['test_samples']}")
print(f"   • Actual Score Range: {data_summary['actual_score_range']} runs")
print(f"   • Predicted Score Range: {data_summary['predicted_score_range']} runs")
print(f"   • Mean Actual: {data_summary['mean_actual']} runs")
print(f"   • Mean Predicted: {data_summary['mean_predicted']} runs")

# Model Equation
print(f"\n🧮 MODEL EQUATION:")
print(f"   {model_report['model_equation']}")

# Feature Importance Summary
print(f"\n📈 FEATURE RANKINGS:")
importance_list = model_report['feature_importance']
for i, feature_info in enumerate(importance_list):
    if feature_info['feature'] != 'intercept':
        rank = i + 1
        print(f"   {rank}. {feature_info['feature']}: {feature_info['coefficient']:.3f}")

print("\n" + "=" * 55)

# Save report to file
with open("model_evaluation_report.json", "w") as f:
    # Convert pandas DataFrames to dicts for JSON serialization
    json_report = model_report.copy()
    json.dump(json_report, f, indent=2, default=str)

print("📁 Report saved to: model_evaluation_report.json")

## 10. Model Diagnostics and Assumptions

In [ ]:
# Check linear regression assumptions
print("🔬 LINEAR REGRESSION ASSUMPTIONS CHECK")
print("=" * 45)

# 1. Linearity - already checked with residuals vs fitted
print(f"\n1️⃣ LINEARITY:")
print(f"   • Check residuals vs predicted plot above")
print(f"   • Random scatter around 0 indicates linearity assumption met")

# 2. Independence - temporal/match independence
print(f"\n2️⃣ INDEPENDENCE:")
print(f"   • Samples from different matches/innings")
print(f"   • Chronological split reduces temporal dependence")
print(f"   • Assumption: ✅ Reasonably met")

# 3. Homoscedasticity - constant variance of residuals
residual_variance_by_fitted = []
fitted_ranges = [(0, 140), (140, 160), (160, 180), (180, 250)]

print(f"\n3️⃣ HOMOSCEDASTICITY (Constant Variance):")
for low, high in fitted_ranges:
    mask = (evaluator.y_pred >= low) & (evaluator.y_pred < high)
    if np.any(mask):
        range_residuals = evaluator.residuals[mask]
        range_var = np.var(range_residuals)
        residual_variance_by_fitted.append(range_var)
        print(f"   • {low}-{high} runs: Residual variance = {range_var:.1f}")

if len(residual_variance_by_fitted) > 1:
    var_ratio = max(residual_variance_by_fitted) / min(residual_variance_by_fitted)
    if var_ratio < 2:
        homoscedasticity_status = "✅ Good - variance is relatively constant"
    elif var_ratio < 4:
        homoscedasticity_status = "⚠️ Moderate - some variance differences"
    else:
        homoscedasticity_status = "❌ Poor - significant variance differences"
    
    print(f"   • Variance ratio: {var_ratio:.2f}")
    print(f"   • Assessment: {homoscedasticity_status}")

# 4. Normality of residuals
from scipy import stats
try:
    shapiro_stat, shapiro_p = stats.shapiro(evaluator.residuals[:1000] if len(evaluator.residuals) > 1000 else evaluator.residuals)
    print(f"\n4️⃣ NORMALITY OF RESIDUALS:")
    print(f"   • Shapiro-Wilk test statistic: {shapiro_stat:.4f}")
    print(f"   • P-value: {shapiro_p:.4f}")
    
    if shapiro_p > 0.05:
        normality_status = "✅ Good - residuals appear normally distributed"
    else:
        normality_status = "⚠️ Violation - residuals not normally distributed"
    
    print(f"   • Assessment: {normality_status}")
    print(f"   • Note: Check histogram in residual analysis above")
except ImportError:
    print(f"\n4️⃣ NORMALITY OF RESIDUALS:")
    print(f"   • Check histogram in residual analysis above")
    print(f"   • Bell-shaped distribution suggests normality")

# 5. No multicollinearity - check VIF if needed
print(f"\n5️⃣ MULTICOLLINEARITY:")
print(f"   • With only 3 features, multicollinearity is less concern")
print(f"   • Features are conceptually distinct (score, wickets, overs)")
print(f"   • Assessment: ✅ Likely not a major issue")

print(f"\n" + "=" * 45)
print(f"Overall: Linear regression assumptions are reasonably well met")
print(f"The model is appropriate for this T20 runs prediction task")

## 11. Model Limitations and Improvements

In [ ]:
print("⚠️ MODEL LIMITATIONS AND IMPROVEMENT OPPORTUNITIES")
print("=" * 60)

r2 = comprehensive_metrics['r2_score']
rmse = comprehensive_metrics['rmse']
within_20 = comprehensive_metrics['within_20_runs']

print(f"\n🚧 CURRENT LIMITATIONS:")

if r2 < 0.8:
    print(f"   • R² = {r2:.3f} - Model explains {r2*100:.1f}% of variance")
    print(f"     → {(1-r2)*100:.1f}% of variance remains unexplained")

if within_20 < 80:
    print(f"   • Only {within_20:.1f}% of predictions within 20 runs")
    print(f"     → Room for improvement in prediction accuracy")

if rmse > 20:
    print(f"   • RMSE = {rmse:.1f} runs is relatively high for T20 cricket")
    print(f"     → Typical errors are substantial relative to T20 scores")

print(f"\n   • Feature Limitations:")
print(f"     → Only 3 basic match-state features")
print(f"     → No player quality, venue, or situational factors")
print(f"     → No powerplay/death overs distinctions")
print(f"     → No recent form or historical performance data")

print(f"\n   • Model Complexity:")
print(f"     → Simple linear relationships may miss non-linear patterns")
print(f"     → No interaction terms between features")
print(f"     → Assumes constant relationships across all match contexts")

print(f"\n🚀 IMPROVEMENT OPPORTUNITIES:")

print(f"\n   📊 Feature Engineering:")
print(f"     • Add run rate features (current RR, required RR)")
print(f"     • Include powerplay/death overs indicators")
print(f"     • Partnership features (current partnership runs/balls)")
print(f"     • Venue-specific adjustments (average scores, conditions)")
print(f"     • Player quality metrics (batting/bowling ratings)")
print(f"     • Recent form indicators")

print(f"\n   🧠 Model Enhancements:")
print(f"     • Polynomial features for non-linear relationships")
print(f"     • Interaction terms (e.g., wickets × overs_remaining)")
print(f"     • Ridge/Lasso regularization for better generalization")
print(f"     • Ensemble methods (Random Forest, Gradient Boosting)")
print(f"     • Phase-specific models (powerplay vs middle vs death)")

print(f"\n   📈 Data Improvements:")
print(f"     • More historical data for training")
print(f"     • Better data quality controls")
print(f"     • External data sources (weather, pitch conditions)")
print(f"     • Real-time feature updates")

print(f"\n   🔧 Technical Enhancements:")
print(f"     • Cross-validation for better model selection")
print(f"     • Hyperparameter tuning")
print(f"     • Model monitoring and retraining pipelines")
print(f"     • A/B testing framework for model comparison")

print(f"\n✅ NEXT STEPS PRIORITY:")
print(f"   1. Add run rate and phase-based features")
print(f"   2. Implement polynomial/interaction terms")
print(f"   3. Try ensemble methods for comparison")
print(f"   4. Validate with recent matches")
print(f"   5. Deploy for real-time testing")

print("\n" + "=" * 60)

## 12. Final Summary and Recommendations

In [ ]:
# Print comprehensive summary
evaluator.print_summary()

# Save evaluation plots
print(f"\n💾 SAVING EVALUATION ARTIFACTS:")
try:
    evaluator.save_plots("../model_evaluation_plots")
    print(f"   ✅ Evaluation plots saved to: ../model_evaluation_plots/")
except Exception as e:
    print(f"   ⚠️ Could not save plots: {e}")

print(f"\n📋 EVALUATION COMPLETE!")
print(f"   • Comprehensive metrics calculated")
print(f"   • Visualizations generated")
print(f"   • Model assumptions checked")
print(f"   • Limitations identified")
print(f"   • Improvement roadmap provided")

print(f"\n🎯 FINAL RECOMMENDATION:")
if r2 >= 0.7 and within_20 >= 70:
    recommendation = "DEPLOY - Model is suitable for production use"
else:
    recommendation = "IMPROVE - Model needs enhancement before deployment"

print(f"   {recommendation}")
print(f"   Continue with feature engineering and model improvements")
print(f"   Consider this as a strong baseline for future development")

print("\n" + "=" * 70)
print("T20 LINEAR REGRESSION MODEL EVALUATION COMPLETE")
print("=" * 70)

## Summary

This comprehensive evaluation of the T20 linear regression model provides:

### Key Findings
- **Performance**: Detailed metrics including R², RMSE, MAE, and accuracy percentages
- **Visualizations**: Multiple plots showing model behavior and error patterns
- **Feature Importance**: Understanding of which factors most influence predictions
- **Model Diagnostics**: Validation of linear regression assumptions

### Artifacts Generated
- Comprehensive evaluation report (JSON)
- Multiple visualization plots
- Performance metrics and insights
- Improvement recommendations

### Next Steps
1. Review the improvement opportunities identified
2. Implement enhanced features and models
3. Compare performance against this baseline
4. Consider deployment based on performance requirements

The model provides a solid foundation for T20 runs prediction with clear pathways for enhancement.